## To-do:

- Post-processing step snapping nearby nodes together. In case of clusters, do that iteratively.
- Ensure we preserve attributes

In [1]:
import logging
import warnings

import matplotlib.pyplot as plt
import momepy
from libpysal import graph
from neatnet import geometry
from neatnet.nodes import consolidate_nodes, remove_false_nodes

from core import algorithms, utils

Filter out the RuntimeWarning showing on Apple Silicon.

In [2]:
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message="invalid value encountered in intersection",
)

Set logging level to debug to see the debugging messages.

In [3]:
# # Get the logger for core.algorithms.simplify
# logger = logging.getLogger('core.algorithms.simplify')
# logger.setLevel(logging.DEBUG)

# # Set the logging format
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # Create a handler for the logger
# handler = logging.StreamHandler()
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(formatter)

# # Add the handler to the logger
# logger.addHandler(handler)

Specify case metadata

In [4]:
case = "Liège"

Read road data

In [5]:
roads = utils.read_no_degree_2(case)

Remove duplicated roads

In [6]:
roads = consolidate_nodes(roads, tolerance=2.1)

Identify artifacts

In [7]:
artifacts, _ = algorithms.simplify.get_artifacts(roads)

Remove edges fully within the artifact (dangles).

In [8]:
_, r_idx = roads.sindex.query(artifacts.geometry, predicate="contains")
roads = roads.drop(roads.index[r_idx])
roads = remove_false_nodes(roads)

Filter only singleton artifacts

In [9]:
rook = graph.Graph.build_contiguity(artifacts, rook=True)

# keep only those artifacts which occur as isolates, i.e. are not part of a larger intersection
artifacts = artifacts.loc[artifacts.index.intersection(rook.isolates)]

Optionally define a subset of data.

In [10]:
data_sample = artifacts.iloc[:50]

Simplify singleton artifacts.

In [11]:
new_roads = algorithms.simplify.simplify_singletons(data_sample, roads)

/Users/martin/miniforge3/envs/simplification/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/martin/miniforge3/envs/simplification/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/martin/miniforge3/envs/simplification/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is tr

Visual exploration of the result.

In [ ]:
for artifact in data_sample.itertuples():
    print(artifact.Index)

    # plot the results for visual verification
    geom = artifact.geometry
    ax = new_roads.clip(geom.buffer(50)).plot(zorder=2, cmap="tab20")
    roads.clip(geom.buffer(50)).plot(
        color="red",
        ax=ax,
        # categorical=True,
        # legend=True,
        linewidth=0.1,
    )
    plt.show()